In [1]:
import pandas as pd
import os 

## Get sector data from Kaggle
https://www.kaggle.com/dgawlik/nyse#securities.csv


In [3]:
sec = pd.read_csv('./securities.csv')
sec.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


### We only need the unique tickers and the GICS Sector

Ref: https://www.fidelity.com/learning-center/trading-investing/markets-sectors/global-industry-classification-standard


In [26]:
sec = sec[['Ticker symbol', 'GICS Sector']]
sec = sec.drop_duplicates()
sec.head()
# list(sec['Ticker symbol'])
pd.unique(sec['GICS Sector'])

array(['Industrials', 'Health Care', 'Information Technology',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Consumer Staples', 'Real Estate', 'Energy',
       'Telecommunications Services'], dtype=object)

### We need to make directories for each sector

In [28]:
sec['new'] = 'key'
sec.head()

,Ticker symbol,GICS Sector,new
0,MMM,Industrials,key
1,ABT,Health Care,key
2,ABBV,Health Care,key
3,ACN,Information Technology,key
4,ATVI,Information Technology,key


In [22]:
# path_to_file = './csv/2017-01-03/'
with pd.HDFStore('./data/book_events_total_view_2017-01-03.h5', 'r') as train:
    for key in train.keys():
        path_to_file = './csv/2017-01-03/'
        if key[1:] in list(sec['Ticker symbol']):
            df = train.get(key)
            df = df[df['book_event_type']=='T']
            # get sector
            sector = sec[sec['Ticker symbol'] == key[1:]]['GICS Sector'].iloc[0]
            path_to_file += sector
            if not os.path.exists(path_to_file):
                os.mkdir(path_to_file)
            
            df.to_csv(path_to_file + '/' + str(key[1:]) +'.csv')